In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=841c82dc89b4cfbc07d8b4fc857514d7b4e1c01b3dfd1aa2564d25ed1826637a
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
import pandas as pd
from google.colab import files
import dateutil.parser
from datetime import datetime,timedelta
from transformers import pipeline
from tqdm.notebook import tqdm
from datetime import datetime
import time
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
uploaded = files.upload() 
bearer_file = files.upload()
_ = files.upload()

def clean_bearer2(file):
  return str(list(file.values())[0])[2:len(str(list(file.values())[0]))-1]


fed_df=pd.read_csv('data_fomc.csv')
fed_df.head() 

Saving twitter_scrap.py to twitter_scrap.py


Saving bearer.txt to bearer.txt


Saving data_fomc.csv to data_fomc.csv


,Date,Text
0,2000-02-02,The Federal Open Market Committee voted today ...
1,2000-03-21,The Federal Open Market Committee voted today ...
2,2000-05-16,The Federal Open Market Committee voted today ...
3,2000-06-28,The Federal Open Market Committee at its meeti...
4,2000-08-22,The Federal Open Market Committee at its meeti...


The fed_df contains FED texts older than Twitter --> delete all the rows with Date before 2006-03-21T00:00Z.

Also considering that Twitter was not so widley used in the first years, let's consiter only the comunication after the 2009 (at that time both washington post and financial times already started to post)

In [ ]:
for index,statement in fed_df.iterrows() :
  if statement[0]>='2009-01-01':
    first_usable=index
    break

new_fed_df=fed_df[first_usable:]
new_fed_df.head()

,Date,Text
72,2009-01-28,The Federal Open Market Committee decided toda...
73,2009-03-18,Information received since the Federal Open Ma...
74,2009-04-29,Information received since the Federal Open Ma...
75,2009-06-24,Information received since the Federal Open Ma...
76,2009-08-12,Information received since the Federal Open Ma...


# **TWEETS DOWNLOAD**

(prossime 2 celle eseguite una volta per tutte e salvando in locale il file tweet.csv)

In [ ]:
import twitter_scrap
timelines=["WSJ","washingtonpost","financialtimes"]
delta=24*2 

id_timeline = {3108351:'WSJ' , 
               2467791 :'washingtonpost' , 
               4898091:'financialtimes' }

In [ ]:
for index,statement in new_fed_df.iterrows() :
  print(index)
  #date conversion to proper format for twitter_scrap.retrive_tweets
  d = dateutil.parser.parse(statement[0])
  start_date=d.strftime('%Y-%m-%dT%H:%M:%S.%Z')+"000Z" 

  #download tweets in "tweets.csv"
  twitter_scrap.retrive_tweets(clean_bearer2(bearer_file),timelines,mr=500,start_time=start_date,delta=delta,filename="tweets.csv")

72
***************************** WSJ **************************
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
# of Tweets added from this response:  47
Total # of Tweets added:  47
-------------------
***************************** washingtonpost **************************
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
# of Tweets added from this response:  30
Total # of Tweets added:  77
-------------------
***************************** financialtimes **************************
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
# of Tweets added from this response:  40
Total # of Tweets added:  117
-------------------
73
***************************** WSJ **************************
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
# of Tweets added from this response:  68
Total # of Tweets added:  68
-------------------
***************************** washingto

# **TEXT SIMILARITY (using the file "tweets.csv" created before)**

In [ ]:
_ = files.upload()#per caricare tweet.csv se non si vuole ogni volta eseguire il download dei tweet alla cella precedente
df=pd.read_csv("tweets.csv")
df.drop(df[df['authorID'] == 'authorID'].index, inplace = True)
df

Saving tweets.csv to tweets.csv


,authorID,created_at,TweetID,Text
0,3108351,2009-01-29 18:30:17+00:00,1159239548,NYSE's New Kid on 'Blocks' http://tinyurl.com/...
1,3108351,2009-01-29 16:30:00+00:00,1158880461,Blagojevich to Speak at Trial http://tinyurl.c...
2,3108351,2009-01-29 15:30:17+00:00,1158704677,Single Layoff Ripples Through Texas Town http:...
3,3108351,2009-01-29 15:00:10+00:00,1158619776,Stocks Stumble Out of Gate http://tinyurl.com/...
4,3108351,2009-01-29 15:00:08+00:00,1158619697,Continuing Jobless Claims Rise http://tinyurl....
...,...,...,...,...
61876,4898091,2022-05-04 02:00:04+00:00,1521670965813559299,BMW Group and Audi have suspended shipments of...
61877,4898091,2022-05-04 01:49:55+00:00,1521668413386813440,Imran Khan rides Pakistan’s economic crisis to...
61878,4898091,2022-05-04 01:49:55+00:00,1521668413248221185,India expands coal mining to beat power crunch...
61879,4898091,2022-05-04 01:49:55+00:00,1521668413071966209,The ghost of Marcos looms over a pivotal Phili...


In [ ]:
import twitter_scrap


JSON_file= defaultdict(lambda: defaultdict(lambda: 0))
relevant_tweets={}
from datetime import datetime,timedelta
last=0

for index,statement in new_fed_df.iterrows() :
  statement_date=statement[0]
  print(index)
  sim_dict = {}
  sim_dict['statement']=statement[1]
  sim_dict['date']=statement[0]
  #list of tweets per statement
  tweets_per_statement=[]

  for i in range (0,len(df)):
    if i<last: 
      continue
    else: 
      tweet_date=datetime.strptime(df.iloc[i]['created_at'], "%Y-%m-%d %H:%M:%S+00:00").strftime('%Y-%m-%d')
      a = datetime.strptime(statement_date, '%Y-%m-%d')+timedelta(days=2)
      b=  datetime.strptime(tweet_date, '%Y-%m-%d')
      if b>a:
        last=i
        break
      tweets_per_statement.append(df.iloc[i])
  
  #Calcolo similarità tra statement[1] e tweets_per_statement
  #print(last,len(tweets_per_statement),tweets_per_statement[0])
  query=statement[1]
  texts=[i[3] for i in tweets_per_statement]
  authorIDs=[i[0] for i in tweets_per_statement]
  tweetIDs=[i[2] for i in tweets_per_statement]

  results=twitter_scrap.similarity(query,texts,authorIDs,tweetIDs)
  #relevant_tweets_per_statement=list(filter(lambda x: x[3] >= 0.5, results))
  #relevant_tweets[statement[0]]=relevant_tweets_per_statement
  #print(len(relevant_tweets_per_statement),len(relevant_tweets))

  converted_list=[]
  for res in results: 
    a=list(res)
    a[3]=float(a[3])
    converted_list.append(a)
  sim_dict['related tweets']=converted_list
  JSON_file[index-72]=sim_dict


72


0it [00:00, ?it/s]

73


0it [00:00, ?it/s]

74


...

0it [00:00, ?it/s]

176


0it [00:00, ?it/s]

177


0it [00:00, ?it/s]

# **SAVING TO JSON FILE**

In [ ]:
import json
# Serializing json
json_object = json.dumps(JSON_file, indent=4)
 
# Writing to sample.json
with open("similarity.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
f = open('similarity.json')
data2 = json.load(f)
f.close()

In [ ]:
for key, value in data2.items() :
    print(key,value['date'],value['related tweets'][0])

0 2009-01-28 ['2467791', '1156841394', 'BREAKING: The House has passed the economic stimulus bill. Update to come.', 0.5270907878875732]
1 2009-03-18 ['3108351', '1348473316', 'Markets Watching the Fed http://bit.ly/Mw5mc', 0.5768254399299622]
2 2009-04-29 ['2467791', '1651840282', 'The Fed sees modest improvements to the economy and leaves short-term interest rates unchanged: http://tr.im/k2UT', 0.5124362111091614]
